In [45]:
DATA='/ahg/regevdata/projects/bn10_cancer_variants/bn10_oana_2018-06-22/results/2018-06-22/results/2020-02-14'

In [46]:
for (fold in c(0)){
    print(fold)
    foldname=paste('fold',as.character(fold),sep='')
    df=read.table(paste(DATA,'/mutfreq/TP53.training_data.',foldname,'.csv',sep=''),
              sep=',',header=TRUE)
    df_train=df[which(df[,foldname]=="True"),]
    print(dim(df_train))
    
    set.seed(1234)
    
    model_def='IARC_Mutation_Counts ~ Signature_1+
    Signature_2+
    Signature_4+
    Signature_5+
    Signature_6+
    Signature_7+
    Signature_13+
    Signature_24+
    A549_p53WT_Nutlin.3_Z.score+
    A549_p53NULL_Nutlin.3_Z.score+
    A549_p53NULL_Etoposide_Z.score'
    model_def=paste(model_def,'+0',sep='')
    myglm <- glm(data=df_train,model_def, family = poisson())
    
    #prediction full model
    df[,'A549_p53WT_Nutlin.3_Z.score']=0
    df[,'A549_p53NULL_Nutlin.3_Z.score']=0
    df[,'A549_p53NULL_Etoposide_Z.score']=0
    pred=predict(myglm,df,type='response')
    df['predicted.mutonly.response']=pred
    
    #prediction only based on mutational signatures
    df_mutonly=df
    df_mutonly[,c('A549_p53WT_Nutlin.3_Z.score',
    'A549_p53NULL_Nutlin.3_Z.score',
    'A549_p53NULL_Etoposide_Z.score')]=0
    pred_mutonly=rowSums(predict(myglm,df_mutonly,type='terms'))#predict(myglm,df_mutonly,type='link')
    df['predicted.mutonly.terms']=pred_mutonly
    
    #write predictions 
    preds=df[,c('Allele','predicted.mutonly.response','predicted.mutonly.terms')]
    write.table(preds,file=paste(DATA,'/mutfreq/TP53.training_data.',foldname,'.pred.csv',sep=''),
               quote=FALSE,row.names=FALSE,col.names=TRUE)

    
}


[1] 0
[1] 2046   47


In [47]:
for (fold in c(0:9)){
    print(fold)
    foldname=paste('fold',as.character(fold),sep='')
    df=read.table(paste(DATA,'/mutfreq/TP53.model_comparison.',foldname,'.csv',sep=''),
                  sep=',',header=TRUE)

    for (modelname in c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut+ sc-eVIP-',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut- sc-eVIP-',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
        print('------------------')
        print(modelname)
        
        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-')){
            
            model_def='IARC_Mutation_Counts ~ Signature_1+
    Signature_2+
    Signature_4+
    Signature_5+
    Signature_6+
    Signature_7+
    Signature_13+
    Signature_24+
        A549_p53WT_Nutlin.3_Z.score+
        A549_p53NULL_Nutlin.3_Z.score+
        A549_p53NULL_Etoposide_Z.score'
        }
        
        if (modelname %in% c('Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            model_def='IARC_Mutation_Counts ~ A549_p53WT_Nutlin.3_Z.score+
        A549_p53NULL_Nutlin.3_Z.score+
        A549_p53NULL_Etoposide_Z.score'   
        }
    
        if (modelname %in% c('Cell- Mut- sc-eVIP+')){
            model_def='IARC_Mutation_Counts ~ HotellingT2'
        }
    
        if (modelname %in% c('Cell- Mut+ sc-eVIP+')){
            model_def='IARC_Mutation_Counts ~ Signature_1+
    Signature_2+
    Signature_4+
    Signature_5+
    Signature_6+
    Signature_7+
    Signature_13+
    Signature_24+
        HotellingT2'
        }


        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
            df_train=df[which(df[,paste('train',foldname,sep='')]=="True"),]
        }

        if (modelname %in% c(
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-',
                        
                        
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            df_train=df[which(df[,foldname]=="True"),]
        }
        
        print(dim(df_train))

        set.seed(1234)
        model_def=paste(model_def,'+0',sep='')
        myglm <- glm(data=df_train, model_def, family = poisson())

        #prediction full model
        pred=predict(myglm,df,type='response')
        #df['predicted']=pred

        #write predictions 
        df[modelname]=pred
        

    }
    write.table(df,file=paste(DATA,'/mutfreq/TP53.model_comparison.',foldname,'.pred.csv',sep=''),
                   quote=FALSE,row.names=FALSE,col.names=TRUE,sep='\t')

}


[1] 0
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(S) test(S)"
[1] 31 49
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(S)"
[1] 1208   50
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(L)"
[1] 1208   51
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(S) test(S)"
[1] 31 52
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(S)"
[1] 1208   53
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(L)"
[1] 1208   54
[1] "------------------"
[1] "Cell- Mut- sc-eVIP+"
[1] 31 55
[1] "------------------"
[1] "Cell- Mut+ sc-eVIP+"
[1] 31 56
[1] 1
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(S) test(S)"
[1] 34 53
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(S)"
[1] 1207   54
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(L)"
[1] 1207   55
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(S) test(S)"
[1] 34 56
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) tes

In [30]:
predict(myglm,df,type='response')

1           2           3           4           5           6 
1229.362696          NA  458.058726  486.736025  231.640606  396.073645 
          7           8           9          10          11          12 
 484.185686   60.239275   23.697139          NA          NA   24.172410 
         13          14          15          16          17          18 
  72.835232  213.609304  115.753626   44.248921   32.589772          NA 
         19          20          21          22          23          24 
 311.002444   13.199618   26.458947  300.022373   67.055455   39.584089 
         25          26          27          28          29          30 
 195.410291          NA   63.439138   48.536149          NA   86.564691 
         31          32          33          34          35          36 
  34.809279  204.442580   63.813001   16.213533   42.392591   46.532772 
         37          38          39          40          41          42 
 773.474501          NA  108.920216 1824.464808   85.943655   91.882069 
         43          44          45          46          47          48 
  29.496629   77.775367  306.599437   74.166397   47.299244   92.294605 
         49          50          51          52          53          54 
 122.976153  109.687469   18.518868          NA  109.897434  224.668194 
         55          56          57          58          59          60 
 306.996396   27.653227  131.628178          NA          NA  200.112344 
         61          62          63          64          65          66 
  64.954895          NA    7.258784          NA   10.897256  168.988949 
         67          68          69          70          71          72 
 169.804156          NA   80.885997   22.949860   51.045411          NA 
         73          74          75          76          77          78 
  91.706849          NA          NA   66.643476          NA          NA 
         79          80          81          82          83          84 
 197.008387   12.734607          NA          NA          NA   16.534326 
         85          86          87          88          89          90 
         NA          NA          NA          NA          NA          NA 
         91          92          93          94          95          96 
         NA   85.773199          NA          NA          NA          NA 
         97          98          99         100         101         102 
         NA   42.434682          NA          NA          NA          NA 
        103         104         105         106         107         108 
  47.599398          NA          NA  100.584274          NA   20.704688 
        109         110         111         112         113         114 
 525.726175          NA          NA          NA          NA          NA 
        115         116         117         118         119         120 
         NA          NA          NA    9.400955          NA          NA 
        121         122         123         124         125         126 
         NA          NA          NA          NA          NA          NA 
        127         128         129         130         131         132 
  56.240650          NA          NA          NA          NA          NA 
        133         134         135         136         137         138 
         NA          NA          NA          NA          NA          NA 
        139         140         141         142         143         144 
         NA          NA          NA          NA          NA          NA 
        145         146         147         148         149         150 
         NA          NA          NA          NA          NA          NA 
        151         152         153         154         155         156 
         NA          NA          NA          NA          NA          NA 
        157         158         159         160         161         162 
         NA          NA          NA          NA          NA          NA 
        163         164         165         166         167         

In [41]:
mtcars

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2


In [42]:
model <- glm(formula= hp ~ wt + disp, data=mtcars, family=poisson)


In [32]:
newdata = data.frame(wt = 2.1, disp = 180)


In [43]:
predict(model, mtcars, type="response")


Mazda RX4       Mazda RX4 Wag          Datsun 710      Hornet 4 Drive 
          113.46500           110.62607            97.12333           151.65884 
  Hornet Sportabout             Valiant          Duster 360           Merc 240D 
          213.32015           131.58851           210.58225           102.25352 
           Merc 230            Merc 280           Merc 280C          Merc 450SE 
          100.52464           107.45788           107.45788           148.42979 
         Merc 450SL         Merc 450SLC  Cadillac Fleetwood Lincoln Continental 
          153.53014           152.76924           265.62795           250.14479 
  Chrysler Imperial            Fiat 128         Honda Civic      Toyota Corolla 
          234.77130            88.54255            92.84407            89.35942 
      Toyota Corona    Dodge Challenger         AMC Javelin          Camaro Z28 
           99.95265           182.21016           174.81221           197.83001 
   Pontiac Firebird           Fiat X1-9       Porsche 914-2        Lotus Europa 
          236.30102            91.00230           103.30685           100.50390 
     Ford Pantera L        Ferrari Dino       Maserati Bora          Volvo 142E 
          212.20400           105.96732           170.64852            97.18374

In [44]:
mtcars$hp

[1] 110 110  93 110 175 105 245  62  95 123 123 180 180 180 205 215 230  66  52
[20]  65  97 150 150 245 175  66  91 113 264 175 335 109

In [37]:
N=2000
prediction=as.integer(N*predict(model, newdata, type="response"))

In [35]:
prediction

1 
472.2163

In [11]:
for (fold in c(0)){
    print(fold)
    foldname=paste('fold',as.character(fold),sep='')
    df=read.table(paste(DATA,'/mutfreq/TP53.training_data.',foldname,'.csv',sep=''),
              sep=',',header=TRUE)
    df_train=df[which(df[,foldname]=="True"),]
    ########
    df_train=df
    print(dim(df_train))
    
    set.seed(1234)
    
    model_def='IARC_Mutation_Counts ~ Signature_1+
    Signature_2+
    Signature_4+
    Signature_5+
    Signature_6+
    Signature_7+
    Signature_13+
    Signature_24+
    A549_p53WT_Nutlin.3_Z.score+
    A549_p53NULL_Nutlin.3_Z.score+
    A549_p53NULL_Etoposide_Z.score'
    model_def=paste(model_def,'+0',sep='')
    myglm <- glm(data=df_train,model_def, family = poisson())
    
    #prediction full model
    df[,'A549_p53WT_Nutlin.3_Z.score']=0
    df[,'A549_p53NULL_Nutlin.3_Z.score']=0
    df[,'A549_p53NULL_Etoposide_Z.score']=0
    pred=predict(myglm,df,type='response')
    df['predicted.mutonly.response']=pred
    
    #prediction only based on mutational signatures
    df_mutonly=df
    df_mutonly[,c('A549_p53WT_Nutlin.3_Z.score',
    'A549_p53NULL_Nutlin.3_Z.score',
    'A549_p53NULL_Etoposide_Z.score')]=0
    pred_mutonly=rowSums(predict(myglm,df_mutonly,type='terms'))#predict(myglm,df_mutonly,type='link')
    df['predicted.mutonly.terms']=pred_mutonly
    
    #write predictions 
    preds=df[,c('Allele','predicted.mutonly.response','predicted.mutonly.terms')]
    write.table(preds,file=paste(DATA,'/mutfreq/TP53.training_data.',foldname,'.pred.csv',sep=''),
               quote=FALSE,row.names=FALSE,col.names=TRUE)
    
    #make predictions for KRAS
    df_kras=read.table('/ahg/regevdata/projects/bn10_cancer_variants/bn10_oana_2018-06-22/results/2018-06-22/results/2019-06-07/KRAS/mutational_signatures/KRAS_mutationalSignatures_Andrew.txt',
    sep='\t',header=TRUE)
    rownames(df_kras)=df_kras[,'Mutation']
    df_kras[,'A549_p53WT_Nutlin.3_Z.score']=0
    df_kras[,'A549_p53NULL_Nutlin.3_Z.score']=0
    df_kras[,'A549_p53NULL_Etoposide_Z.score']=0
    
    
    df_kras['predicted.mutonly.response']=predict(myglm,df_kras,type='response')
    pred_kras=rowSums(predict(myglm,df_kras,type='terms',terms=c('Signature_1',
                                           'Signature_2',
                                           'Signature_4',
                                           'Signature_5',
                                           'Signature_6',
                                           'Signature_7',
                                           'Signature_13',
                                           'Signature_24')))
    
    df_kras['predicted.mutonly.terms']=pred_kras
    write.table(df_kras,file=paste(DATA,'/mutfreq/KRAS.predictedFromTP53.',foldname,'.pred.csv',sep=''),
               quote=FALSE,row.names=FALSE,col.names=TRUE)
    
}



[1] 0
[1] 2414   47


In [9]:
for (fold in c(0:9)){
    print(fold)
    foldname=paste('fold',as.character(fold),sep='')
    df=read.table(paste(DATA,'/mutfreq/TP53.model_comparison.',foldname,'.csv',sep=''),
                  sep=',',header=TRUE)

    for (modelname in c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut+ sc-eVIP-',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut- sc-eVIP-',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
        print('------------------')
        print(modelname)
        
        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-')){
            
            model_def='IARC_Mutation_Counts ~ mutational_signature_score+
        A549_p53WT_Nutlin.3_Z.score+
        A549_p53NULL_Nutlin.3_Z.score+
        A549_p53NULL_Etoposide_Z.score'
        }
        
        if (modelname %in% c('Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            model_def='IARC_Mutation_Counts ~ A549_p53WT_Nutlin.3_Z.score+
        A549_p53NULL_Nutlin.3_Z.score+
        A549_p53NULL_Etoposide_Z.score'   
        }
    
        if (modelname %in% c('Cell- Mut- sc-eVIP+')){
            model_def='IARC_Mutation_Counts ~ HotellingT2'
        }
    
        if (modelname %in% c('Cell- Mut+ sc-eVIP+')){
            model_def='IARC_Mutation_Counts ~ mutational_signature_score+
        HotellingT2'
        }


        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
            df_train=df[which(df[,paste('train',foldname,sep='')]=="True"),]
        }

        if (modelname %in% c(
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-',
                        
                        
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            df_train=df[which(df[,foldname]=="True"),]
        }
        
        print(dim(df_train))

        set.seed(1234)
        myglm <- glm(data=df_train, model_def, family = poisson())

        #prediction full model
        pred=predict(myglm,df,type='response')
        #df['predicted']=pred

        #write predictions 
        df[modelname]=pred
        

    }
    write.table(df,file=paste(DATA,'/mutfreq/TP53.model_comparison.',foldname,'.pred.csv',sep=''),
                   quote=FALSE,row.names=FALSE,col.names=TRUE,sep='\t')

}


[1] 0
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(S) test(S)"
[1] 40 50
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(S)"
[1] 700  51
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(L)"
[1] 700  52
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(S) test(S)"
[1] 40 53
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(S)"
[1] 700  54
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(L)"
[1] 700  55
[1] "------------------"
[1] "Cell- Mut- sc-eVIP+"
[1] 40 56
[1] "------------------"
[1] "Cell- Mut+ sc-eVIP+"
[1] 40 57
[1] 1
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(S) test(S)"
[1] 37 54
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(S)"
[1] 697  55
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(L)"
[1] 697  56
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(S) test(S)"
[1] 37 57
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(S)"
[1] 69

In [12]:
#KRAS models

for (fold in c(0:9)){
    print(fold)
    foldname=paste('fold',as.character(fold),sep='')
    df=read.table(paste(DATA,'/mutfreq/KRAS.model_comparison.',foldname,'.csv',sep=''),
                  sep=',',header=TRUE)

    for (modelname in c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut+ sc-eVIP-',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut- sc-eVIP-',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
        print('------------------')
        print(modelname)
        
        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-')){
            
            model_def='COSMIC.1 ~ mutational_signature_score+
        GILA_day7'
        }
        
        if (modelname %in% c('Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            model_def='COSMIC.1 ~ GILA_day7'   
        }
    
        if (modelname %in% c('Cell- Mut- sc-eVIP+')){
            model_def='COSMIC.1 ~ HotellingT2'
        }
    
        if (modelname %in% c('Cell- Mut+ sc-eVIP+')){
            model_def='COSMIC.1 ~ mutational_signature_score+
        HotellingT2'
        }


        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
            df_train=df[which(df[,paste('train',foldname,sep='')]=="True"),]
        }

        if (modelname %in% c(
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-',
                        
                        
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            df_train=df[which(df[,foldname]=="True"),]
        }
        
        print(dim(df_train))

        set.seed(1234)
        myglm <- glm(data=df_train, paste(model_def,'+0',sep=''), 
                     family = poisson())

        #prediction full model
        pred=predict(myglm,df,type='response')
        #df['predicted']=pred

        #write predictions 
        df[modelname]=pred
        

    }
    write.table(df,file=paste(DATA,'/mutfreq/KRAS.model_comparison.',foldname,'.pred.csv',sep=''),
                   quote=FALSE,row.names=FALSE,col.names=TRUE,sep='\t')

}

[1] 0
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(S) test(S)"
[1] 24 94
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(S)"
[1] 84 95
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(L)"
[1] 84 96
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(S) test(S)"
[1] 24 97
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(S)"
[1] 84 98
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(L)"
[1] 84 99
[1] "------------------"
[1] "Cell- Mut- sc-eVIP+"
[1]  24 100
[1] "------------------"
[1] "Cell- Mut+ sc-eVIP+"
[1]  24 101
[1] 1
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(S) test(S)"
[1] 29 94
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(S)"
[1] 85 95
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(L)"
[1] 85 96
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(S) test(S)"
[1] 29 97
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(S)"
[1] 85 98
[1] 

In [ ]:
for (fold in c(0:9)){
    print(fold)
    foldname=paste('fold',as.character(fold),sep='')
    df=read.table(paste(DATA,'/mutfreq/KRAS.model_comparison.',foldname,'.csv',sep=''),
                  sep=',',header=TRUE)

    for (modelname in c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut+ sc-eVIP-',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut- sc-eVIP-',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
        print('------------------')
        print(modelname)
        
        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-')){
            
            model_def='COSMIC.1 ~ Signature_1+
    Signature_2+
    Signature_3+
    Signature_4+
    Signature_5+
    Signature_6+
    Signature_7+
    Signature_8+
    Signature_9+
    Signature_10+
    Signature_11+
    Signature_12+
    Signature_13+
    Signature_14+
    Signature_15+
    Signature_16+
    Signature_17+
    Signature_18+
    Signature_19+
    Signature_20+
    Signature_21+
    Signature_22+
    Signature_23+
    Signature_24+
    Signature_25+
    Signature_26+
    Signature_27+
    Signature_28+
    Signature_29+
    Signature_30+
        GILA_day7'
        }
        
        if (modelname %in% c('Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            model_def='COSMIC.1 ~ GILA_day7'   
        }
    
        if (modelname %in% c('Cell- Mut- sc-eVIP+')){
            model_def='COSMIC.1 ~ HotellingT2'
        }
    
        if (modelname %in% c('Cell- Mut+ sc-eVIP+')){
            model_def='COSMIC.1 ~ Signature_1+
    Signature_2+
    Signature_3+
    Signature_4+
    Signature_5+
    Signature_6+
    Signature_7+
    Signature_8+
    Signature_9+
    Signature_10+
    Signature_11+
    Signature_12+
    Signature_13+
    Signature_14+
    Signature_15+
    Signature_16+
    Signature_17+
    Signature_18+
    Signature_19+
    Signature_20+
    Signature_21+
    Signature_22+
    Signature_23+
    Signature_24+
    Signature_25+
    Signature_26+
    Signature_27+
    Signature_28+
    Signature_29+
    Signature_30+
        HotellingT2'
        }


        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
            df_train=df[which(df[,paste('train',foldname,sep='')]=="True"),]
        }

        if (modelname %in% c(
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-',
                        
                        
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            df_train=df[which(df[,foldname]=="True"),]
        }
        
        print(dim(df_train))

        set.seed(1234)
        myglm <- glm(data=df_train, model_def, family = poisson())

        #prediction full model
        pred=predict(myglm,df,type='response')
        #df['predicted']=pred

        #write predictions 
        df[modelname]=pred
        

    }
    write.table(df,file=paste(DATA,'/mutfreq/KRAS.model_comparison.',foldname,'.pred.csv',sep=''),
                   quote=FALSE,row.names=FALSE,col.names=TRUE,sep='\t')

}